In [0]:
!pip install bigartm

     |████████████████████████████████| 1.9MB 3.5MB/s 


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ls /gdrive/My\ Drive/Datasets/r8/lemmatized_wo_stopwords

test.bz2  train.bz2


In [0]:
!mkdir r8

# BigARTM. Руководство для пользователей Python API.

Автор - **Мурат Апишев** (great-mel@yandex.ru)

Этот ноутбук представляет собой руководство по использованию библиотеки в основных случаях использования. Свои вопросы по случаям, не рассмотренным в данном документе, присылайте в сообщество bigartm-users@googlegroups.com.

Предполагается, что Вы в точности выполнили инструкции по установке библиотеки и её настройки для использования из Python (http://bigartm.readthedocs.org/en/master/installation/index.html) и модуль artm у Вас импортируется без ошибок.

Итак, импортируем этот модуль:

In [0]:
import artm

print(artm.version())

0.9.2


Каждый из описанных далее сценариев является отдельным блоком кода, не зависящим от других (за исключением самого первого, про словари и батчи - его надо выполнять всегда). Код рабочий (в том смысле его можно копипастить в свои скрипты), в том случае, если Вы верно подготовите все требуемые данные и разместите их там, где нужно.

Прежде, чем задавать вопросы по указанному выше адресу, убедись в том, что он не связан с Вашими некорректными действиями. Одна из наиболее типичных ошибок имеет примерно такой вид:

DiskReadException: File vocab.kos.txt does not exist.

Скорее всего, она связана с тем, что Вы не подготовили данные, неверно разместили их, либо неверно назвали файл с ними (дважды написали расширение или что-то в этом роде).

Также предполагается, что Вы владеете языком Python на достаточном уровне, вопросы по особенностям языка лучше задавать на соответствующих интернет-ресурсах.

### Словари и батчи в BigARTM

In [0]:
import os
import bz2
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from gensim.models.phrases import Phrases, Phraser

/usr/local/lib/python3.6/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [0]:
def to_vw_format(df, out_file, text_col='text', ngram_col='ngrams', label_col='label'):
    '''
    Format example: doc_100500 |@default_class aaa:2 bbb:4 ccc ddd:6 |@labels_class class_1 class_6
    '''
    with open(out_file, 'w') as f:
        for index, row in df.iterrows():
            line = f'{index} |@default_class {row[text_col]} |@ngrams_class {row[ngram_col]}'
            if label_col is not None:
                line += f' |@labels_class {row[label_col]}'
            f.write(line + '\n')

In [0]:
def get_bigrams(senntence):
    """
    Фильтруем токены и оставляем только биграммы
    """
    tokens = senntence.split()
    tokens_bigrams = bigram[tokens]
    return ' '.join([x for x in tokens_bigrams if '_' in x])

In [0]:
# def load_and_save_vw(dataset_path):
#     with bz2.BZ2File(dataset_path, "r") as f:
#         df = pd.read_json(f, lines=True)
#         to_vw_format(df, os.path.join(dataset, 'train_vw.txt'))

Прежде, чем приступать непосредственно к моделированию, необходимо привести данные к формату, подходящему для использования библиотекой. Сперва ознакомьтесь с форматами сырых данных, которые можно подавать BigARTM (http://bigartm.readthedocs.org/en/master/formats.html). Задача подготовки файла в одном из таких форматах лежит на Вас. Перевод же этих данных во внутренний формат библиотеки (пакеты документов, именуемы батчами), можно проделать с помощью создания объекта класса BatchVectorizer.

Впрочем, есть один более простой вариант обработки Вашей коллекции на тот случай, если она не слишком велика и Вам не нужно сохранять её в батчи. Для этого Вам необходимо получить для своей коллекции переменную n\_wd типа numpy.ndarray размера "число уникальных слова в коллекции" на "число документов", содержащую счётчики $n_{wd}$ (т. е. матрицу "мешка слов") и Python dict vocabulary в котором ключом является индекс строки этой матрицы, а значением - исходное слово. Получить такие переменные при наличии у Вас сырых текстов проще всего с использованием CountVectorizer (или схожих классов) из sklearn.

При наличии этих переменных можно запустить следующий код:

# Подготовка батчей в памяти

In [0]:
base_path = '/gdrive/My Drive/Datasets/'
dataset = 'r8'
os.makedirs(dataset, exist_ok=True)

In [0]:
os.path.join(base_path, dataset, 'lemmatized_wo_stopwords/train.bz2')

In [0]:
with bz2.BZ2File(os.path.join(base_path, dataset, 'lemmatized_wo_stopwords/train.bz2'), "r") as f:
    df_train = pd.read_json(f, lines=True)

In [0]:
df_train.head()

,text,label
0,champion product ch approve stock split champi...,1
1,computer terminal system cpml complete sale co...,2
2,cobanco inc cbco year net shr ct vs dlr net vs...,1
3,international inc nd qtr jan oper shr loss two...,1
4,brown forman inc bfd th qtr net shr one dlr vs...,1


In [0]:
df_train['label'].value_counts(normalize=True)

1    0.517776
2    0.290975
6    0.046126
3    0.045761
8    0.037557
7    0.034640
4    0.019690
5    0.007475
Name: label, dtype: float64

In [0]:
with bz2.BZ2File(os.path.join(base_path, dataset, 'lemmatized_wo_stopwords/test.bz2'), "r") as f:
    df_test = pd.read_json(f, lines=True)

# Составление ngrams

In [0]:
phrases = Phrases(df_train['text'].str.split(), min_count=10, threshold=1)
bigram = Phraser(phrases)

In [0]:
get_bigrams(df_train['text'].str.split().iloc[0])

['stock_split',
 'product_inc',
 'say_board',
 'two_one',
 'stock_split',
 'common_share',
 'shareholder_record',
 'company_also',
 'say_board',
 'shareholder_annual',
 'five_mln']

In [0]:
df_train['ngrams'] = df_train['text'].apply(get_bigrams)
df_test['ngrams'] = df_test['text'].apply(get_bigrams)

# Сохранение в Vowpal Wabbit формат

In [0]:
to_vw_format(df_train, os.path.join(dataset, 'train_vw.txt'))
to_vw_format(df_test, os.path.join(dataset, 'test_vw.txt'), label_col=None)

In [0]:
!head r8/train_vw.txt

0 |@default_class champion product ch approve stock split champion product inc say board director approve two one stock split common share shareholder record april company also say board vote recommend shareholder annual meeting april increase authorized capital stock five mln mln share reuter |@ngrams_class stock_split product_inc say_board two_one stock_split common_share shareholder_record company_also say_board shareholder_annual five_mln |@labels_class 1
1 |@default_class computer terminal system cpml complete sale computer terminal system inc say complete sale share common stock warrant acquire additional one mln share sedio n v lugano switzerland dlr company say warrant exercisable five year purchase price dlr per share computer terminal say sedio also right buy additional share increase total holding pct computer terminal outstanding common stock certain circumstance involve change control company company say condition occur warrant would exercisable price equal pct common stoc

Встроенный парсер библиотеки преобразовал Ваши данные в батчи, обернув их в объект класса BatchVectorizer, который является универсальным типом входных данных для всех методов Python API, прочесть о нём можно тут http://bigartm.readthedocs.org/en/master/python_interface/batches_utils.html. Сами батчи разместились в директории, которую Вы указали как target_folder.

Если же у Вас есть файл в формате Vowpal Wabbit, то воспользуйтесь следующим кодом:

In [0]:
batches_path = os.path.join(dataset, 'batches')
batch_vectorizer = artm.BatchVectorizer(data_path='r8/train_vw.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder=batches_path)

batches_test_path = os.path.join(dataset, 'batches_test')
batch_vectorizer_test = artm.BatchVectorizer(data_path='r8/test_vw.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder=batches_test_path)

Следующая цель после создания батчей - создание словаря. Они хранят информацию обо всех уникальных словах в коллекции. Словарь создаётся вне модели, и различными способами (посмотреть их все Вы можете вот тут http://bigartm.readthedocs.org/en/master/python_interface/dictionary.html).
Самый базовый вариант - "собрать" словарь по директории с батчами. Это нужно делать один раз в самом начале работы с новой коллекцией следующим образом:

In [0]:
dictionary = artm.Dictionary()
dictionary.gather(data_path=batches_path)

In [0]:
dictionary.save_text(os.path.join(dataset, 'vocab.txt'))

Последний момент: все методы создания BatchVectorizer автоматически генерируют словарь по умолчанию, доступ к которому можно получить, написав:

In [0]:
batch_vectorizer.dictionary

artm.Dictionary(name=bf77a61a-7e4d-466e-9ba1-971896744115, num_entries=18541)

Если Вам это, по каким-то причинам, не нужно, при создании BatchVectorizer нужно добавить параметр gather_dictionary=False.
Этот флаг будет проигнорирован только в случае data_format равного n\_wd, поскольку в этом случае другого способа получить словарь не существует.

### Раздел 3: построение мультимодальной тематической модели с регуляризацией и оцениванием качества; метод ARTM.transform().

Теперь перейдём к более сложным случаям. В прошлом разделе было упомянуто понятие модальности. Это нечто, соответствующее каждому слову. Я бы определил это, как вид слова. Бывают слова текста, бывают слова, из которых состоит заголовок. А также слова-имена авторов текста, и даже картинку, если её перекодировать в текст, можно считать набором слов, и слова эти будут типа "слова из которых состоит картинка". И таких видов слов можно придумать очень много.

Так вот, в BigARTM каждое слово имеет тип модальность. Обозначается она не интуитивно - class_id. Ничего общего с классификацией это не имеет, просто неудачное название, которое уже поздно менять. У каждого слова есть такой class_id, Вы всегда можете задать его сами, либо же библиотека автоматически задаст class_id = '@default_class' (если Вы смотрели внутрь словарей, то, наверняка, видели эту конструкцию). Это - тип обычных слов, тип по умолчанию.

В большинстве случаев модальности не потребуются, но есть такие ситуации, когда они незаменимы. Например, при классификации документов. Собственно, именно этот пример мы и рассмотрим.

Все данные придётся пересоздавать с учётом наличия модальностей. От Вас потребуется создание файла в формате Vowpal Wabbit, в котором каждая строка - это документ, а каждый документ состоит из обычных слов и слов-меток классов, к которым относится документ. 

Пример:
doc_100500 |@default_class aaa:2 bbb:4 ccc ddd:6 |@labels_class class_1 class_6

Всё это подробно описано здесь http://bigartm.readthedocs.org/en/master/formats.html.

Теперь проделайте с этим файлом все необходимые манипуляции из вводной части, чтобы получить нужные батчи и словарь.

Далее, Вам надо объяснить модели, какие у Вас есть модальности, и какие степени влияния на модель Вы хотите им задать. Степень влияния - это коэффициент модальности $\tau_m$ (об этом Вы также должны иметь представление). Модель по умолчанию использует только слова модальности '@default_class' и её берёт с $\tau_m$ = 1.0. Хотите использовать другие модальности и веса - надо задать эти требования в конструкторе модели следующим кодом:

In [0]:
model = artm.ARTM(num_topics=20, class_ids={'@default_class': 3.0, '@ngrams_class': 3.0, '@labels_class': 1.0}, seed=42)

Итак, мы попросили модель учитывать эти две модальности, причём метки классов сделать в 5 раз более влиятельными, чем обычные слова. Отмечу, что если в вашем файле с данными были ещё модальности, а Вы их тут не отметили - они не будут учтены. Опять же, если Вы отметите в конструкторе модальности, которых нет в данных - случится то же самое.

Разумеется, поле class_ids, как и все остальные, является переопределяемым, Вы всегда можете изменить веса модальностей:

Обновлять веса надо именно так, задавая весь словарь, не надо пытаться обратиться по ключу к отдельной модальности, class_ids обновляется с помощью словаря, но сама словарём не является (словарь - в смысле Python dict).

При следующем запуске fit_offline() или fit_online() эта информация будет учтена.

Теперь к модели надо подключить регуляризаторы и метрики качества. Весь это процесс уже был рассмотрен, за исключением одного момента. Все метрики на матрице $\Phi$ (а также перплексия) и регуляризаторы $\Phi$ имеют поля для работы модальностями. Т. е. в этих полях Вы можете определить, с каким модальностями метрика/регуляризатор должна работать, остальные будут проигнорированы (по аналогии с полем topic_names для тем).

Поле модальности может быть либо class_id, либо class_ids. Первое - это строка с именем одной модальности, с которой надо работать, второе - список строк с такими модальностями.

**Важный момент** со значениями по умолчанию. Для class_id отсутствие заданного Вами значения означает class_id = '@default_class'. Для class_ids отсутствие значения означает использование всех имеющихся в модели модальностей.

Посмотреть информацию детально о каждой метрике и каждом регуляризаторе можно по уже данным ранее ссылках http://bigartm.readthedocs.org/en/master/python_interface/regularizers.html и http://bigartm.readthedocs.org/en/master/python_interface/scores.html.

Давайте добавим в модель метрику разреженности $\Phi$ для модальности меток классов, а также регуляризаторы декорреляции тем для каждой из модальностей, после чего запустим процесс обучения модели:

In [0]:
model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score', class_id='@labels_class'))

model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_def', class_ids=['@default_class']))
model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_ngram', class_ids=['@ngrams_class']))
model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_lab', class_ids=['@labels_class']))

In [0]:
model.initialize(dictionary)

In [0]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

Итак, работы по дальнейшей настройке модели, подбору коэффициентов регуляризации, весов модальностей и просмотра метрик остаются Вам. Сейчас же перейдём к использованию обученной модели для классификации тестовых данных.

Напомню, что в задаче классификации у Вас есть обучающие данные (коллекция, на которой Вы тренировали модель, где для каждого документа модели известны его метки классов) и данные тестовые, метки которых известны Вам, но не сообщаются модели. Эти скрытые метки модель и должна предсказать, глядя на тестовые документы, а Ваша задача - взять верные ответы и то, что Вам выдала модель, и сравнить, например, посчитав AUC.

Подсчёт AUC или ещё каких либо метрик - это Ваше дело, мы этого касаться не будем. А мы займёмся получением для новых документов векторов p(c|d) длиной в количество классов, где каждый элемент - вероятность класса c для данного документа d.

Итак, у нас имеется модель. Предполагается, что тестовые документы были убраны в отдельный файл в формате Vowpal Wabbit, на основе которого Вы сумели сгенерировать батчи, которые описываются переменной batch_vectorizer_test (см. вводный раздел). Также предполагается, что сохранили Вы тестовые батчи в отдельную директорию (не в ту, в которую сохранялись обучающие батчи).

Ваши тестовые документы не должны содержать информацию о метках классов (а именно: в тестовом файле не должно быть строки '|@labels_class'), также тестовые документы не содержат слов, которых не было в документах обучающих, иначе такие слова будут проигнорированы.

Если все эти условия выполнены, можно переходить к использованию ARTM.transform() (об этом написано тут http://bigartm.readthedocs.org/en/master/python_interface/artm.html), который позволяет для всех документов из данного объекта BatchVectorizer получить матрицу вероятностей p(t|d) (т. е. $\Theta$), либо матрицу p(c|d) для любой указанной модальности.

Чтобы получить $\Theta$ делаем так:

In [0]:
theta_test = model.transform(batch_vectorizer=batch_vectorizer_test)

А чтобы получить p(c|d), запустите такой код:

In [0]:
p_cd_test = model.transform(batch_vectorizer=batch_vectorizer_test, predict_class_id='@labels_class')

In [0]:
p_cd_test = p_cd_test.T

In [0]:
y_pred = p_cd_test.idxmax(axis=1).astype(int)

In [0]:
print(f"f1_macro = {f1_score(df_test['label'], y_pred, average='macro')}, \
    accuracy = {accuracy_score(df_test['label'], y_pred)}")

f1_macro = 0.13839456319573723,     accuracy = 0.38784833257195067


Таким образом, Вы получили предсказания модели в pandas.DataFrame. Теперь Вы можете оценить степень качества предсказаний построенной Вами модели любым способом, который Вам необходим.

# Подбор параметров

In [0]:
import itertools
import multiprocessing

In [0]:
def fit_model(class_ids):
    model = artm.ARTM(num_topics=20, class_ids=class_ids, seed=42)
    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score', class_id='@labels_class'))

    for class_id in class_ids.keys():
        model.regularizers.add(artm.DecorrelatorPhiRegularizer(name=f'decorrelator_phi_{class_id.strip("@")}', \
                                                               class_ids=[class_id]))
        
    model.initialize(dictionary)
    model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

    p_cd_test = model.transform(batch_vectorizer=batch_vectorizer_test, predict_class_id='@labels_class').T
    y_pred = p_cd_test.idxmax(axis=1).astype(int)
    return f1_score(df_test['label'], y_pred, average='macro')

def get_all_combinations(d):
    keys = d.keys()
    values = (d[key] for key in keys)
    combinations = [dict(zip(keys, combination)) for combination in itertools.product(*values)]
    return combinations

In [0]:
params = {
    '@default_class': list(range(5)), 
    '@ngrams_class': [1.0], 
    '@labels_class': [1.0]
}

In [0]:
# parallel version
# pool = multiprocessing.Pool(multiprocessing.cpu_count())
# param_combinations = get_all_combinations(params)
# scores = pool.map(fit_model, param_combinations)

In [0]:
best_params, best_score = {}, 0
for param in tqdm(get_all_combinations(params), position=0):
    score = fit_model(param)
    if score > best_score:
        best_score = score
        best_params = param

100%|██████████| 5/5 [02:10<00:00, 26.11s/it]

In [0]:
best_score

0.13440685065078858

In [0]:
best_params

{'@default_class': 1, '@labels_class': 1.0, '@ngrams_class': 1.0}